### This uses the latest SToRM Tuner package from this web site
https://github.com/ben-arnao/stochasticmutatortuner
It is Open Source and it is excellent in speed and versatility!

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential
#from tuner import Tuner

In [2]:
from storm.tuner import Tuner

In [3]:
datapath = '.'
sep=','
filename = 'breast_cancer.csv'
df=pd.read_csv(datapath+filename, sep=sep)
df['diagnosis'] = df['diagnosis'].map({'B':0,'M':1})
print(df.shape)
df.head()

(569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,893783,0,11.70,19.11,74.33,418.7,0.08814,0.05253,0.015830,0.011480,...,12.61,26.55,80.92,483.1,0.12230,0.10870,0.079150,0.05741,0.3487,0.06958
1,898431,1,19.68,21.68,129.90,1194.0,0.09797,0.13390,0.186300,0.110300,...,22.75,34.66,157.60,1540.0,0.12180,0.34580,0.473400,0.22550,0.4045,0.07918
2,854039,1,16.13,17.88,107.00,807.2,0.10400,0.15590,0.135400,0.077520,...,20.21,27.26,132.70,1261.0,0.14460,0.58040,0.527400,0.18640,0.4270,0.12330
3,901034302,0,12.54,18.07,79.42,491.9,0.07436,0.02650,0.001194,0.005449,...,13.72,20.98,86.82,585.7,0.09293,0.04327,0.003581,0.01635,0.2233,0.05521
4,91550,0,11.74,14.69,76.31,426.0,0.08099,0.09661,0.067260,0.026390,...,12.45,17.60,81.25,473.8,0.10730,0.27930,0.269000,0.10560,0.2604,0.09879


In [4]:
X=df.iloc[:,2:] ## independent features
y=df.iloc[:,1] ## dependent features
X.shape, y.shape

((569, 30), (569,))

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
def build_model(hp):
    model = Sequential()

    # example of model-wide unordered categorical parameter
    activation_fn = hp.Param('activation', ['relu', 'selu', 'elu'])

    # example of per-block parameter
    model.add(Dense(hp.Param('kernel_size_' + str(0), 
                             sorted(np.linspace(64,200,100).astype(int),reverse=True), 
                             ordered=True)))

    model.add(Activation(activation_fn))

    # example of boolean param
    if hp.Param('use_batch_norm', [True, False]):
        model.add(BatchNormalization())

    if hp.Param('use_dropout', [True, False]):

        # example of nested param
        #
        # this param will not affect the configuration hash, if this block of code isn't executed
        # this is to ensure we do not test configurations that are functionally the same
        # but have different values for unused parameters
        model.add(Dropout(hp.Param('dropout_value', [0.1, 0.2, 0.3, 0.4, 0.5], ordered=True)))
    kernel_size =  hp.values['kernel_size_' + str(0)]
    # example of inline ordered parameter
    for x in range(hp.Param('num_layers', [1, 2, 3], ordered=True)):

        kernel_size = int(0.75*kernel_size)
        # example of per-block parameter
        model.add(Dense(kernel_size))
        
        model.add(Activation(activation_fn))

        # example of boolean param
        if hp.Param('use_batch_norm', [True, False]):
            model.add(BatchNormalization())

        if hp.Param('use_dropout', [True, False]):

            # example of nested param
            #
            # this param will not affect the configuration hash, if this block of code isn't executed
            # this is to ensure we do not test configurations that are functionally the same
            # but have different values for unused parameters
            model.add(Dropout(hp.Param('dropout_value', [0.1, 0.2, 0.3, 0.4, 0.5], ordered=True)))
    output_activation = 'sigmoid'
    num_predicts = 2
    model.add(Dense(num_predicts, activation=output_activation))

    loss_fn = 'sparse_categorical_crossentropy'
    model.compile(loss=loss_fn, optimizer='Adam', metrics=['accuracy'])
    return model

from tensorflow.keras import callbacks

def custom_score_function(x_train, y_train, x_valid, y_valid, model, batch_size,):
    
    es = callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.00001, patience=10,
                        verbose=0, mode='max', baseline=None, restore_best_weights=True)


    history = model.fit(x_train, y_train, epochs=25, batch_size=batch_size, 
                        validation_data=(x_valid,y_valid), callbacks=[es],
                        verbose=0)
    # here we can defined custom logic to assign a score to a configuration
    return np.mean(history.history['loss'][-5:])

class MyTuner(Tuner):

    def run_trial(self, trial, X_train, y_train, X_test, y_test, 
                  ):
        hp = trial.hyperparameters
        model = build_model(hp)
        # here we can access params generated in the builder function
        # example of supplementary paramteter that will be accessed elsewhere  ##
        batch_limit = int(min(2000, X_train.shape[0]))
        batch_nums = int(max(200, X_train.shape[0]))
        batch_size = hp.Param('batch_size', sorted(np.linspace(32,
                                batch_limit,batch_nums).astype(int),reverse=True),
                                 ordered=True)

        score = custom_score_function(X_train,
                                      y_train,
                                      X_test,
                                      y_test,
                                      model=model,
                                      batch_size=batch_size,
                                     )
        self.score_trial(trial, score)

In [7]:
tuner = MyTuner(project_dir='C:/',
                build_fn=build_model,
                objective_direction='min',
                init_random=5,
                max_iters=10,
                randomize_axis_factor=0.5,
                overwrite=True)

new tuner initialized


In [8]:
# parameters passed through 'search' go directly to the 'run_trial' method
tuner.search(X_train, y_train, X_test, y_test,)


-------- OPTIMIZING HYPERPARAMETERS --------

<><><> NEW BEST! <><><>
	activation : relu
	batch_size : 383
	dropout_value : 0.4
	kernel_size_0 : 176
	num_layers : 3
	use_batch_norm : True
	use_dropout : True
1 | score: 0.39704698
2 | score: 0.4659237
3 | score: 30.38740196
<><><> NEW BEST! <><><>
	activation : relu
	batch_size : 325
	kernel_size_0 : 66
	num_layers : 2
	use_batch_norm : True
	use_dropout : False
4 | score: 0.19108576
5 | score: 10.6162756
6 | score: 0.30448833
<><><> NEW BEST! <><><>
	activation : elu
	batch_size : 325
	kernel_size_0 : 66
	num_layers : 2
	use_batch_norm : True
	use_dropout : False
7 | score: 0.16273599
<><><> NEW BEST! <><><>
	activation : elu
	batch_size : 325
	kernel_size_0 : 66
	num_layers : 1
	use_batch_norm : True
	use_dropout : False
8 | score: 0.1547011
9 | score: 0.5016996
10 | score: 0.26583421
11 | score: 0.1771562
tuner finished


In [9]:
model = build_model(tuner.get_best_config())

In [10]:
patience = 10
val_monitor = 'val_loss'
lr_patience = max(2,int(patience*1.5))
rlr = callbacks.ReduceLROnPlateau(monitor=val_monitor, factor=0.25,
                patience=lr_patience, min_lr=1e-6, mode='auto', min_delta=0.00001, 
                                  cooldown=0, verbose=1)
es = callbacks.EarlyStopping(monitor=val_monitor, min_delta=0.00001, patience=10,
                    verbose=0, mode='min', baseline=None, restore_best_weights=True)
callbacks_list = [es, rlr]

In [11]:
NUMBER_OF_EPOCHS = 50
STEPS_PER_EPOCH = int(max(2,NUMBER_OF_EPOCHS/10))
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                epochs=NUMBER_OF_EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
                callbacks=callbacks_list, validation_steps=STEPS_PER_EPOCH,
               shuffle=False)

Epoch 1/50
5/5 [==============================] - 1s 68ms/step - loss: 0.5519 - accuracy: 0.6809 - val_loss: 2.7587 - val_accuracy: 0.3743
Epoch 2/50
5/5 [==============================] - 0s 15ms/step - loss: 0.2168 - accuracy: 0.9115 - val_loss: 2.5830 - val_accuracy: 0.3860
Epoch 3/50
5/5 [==============================] - 0s 14ms/step - loss: 0.1844 - accuracy: 0.9232 - val_loss: 2.4345 - val_accuracy: 0.3918
Epoch 4/50
5/5 [==============================] - 0s 17ms/step - loss: 0.1794 - accuracy: 0.9252 - val_loss: 2.1829 - val_accuracy: 0.3918
Epoch 5/50
5/5 [==============================] - 0s 14ms/step - loss: 0.1736 - accuracy: 0.9303 - val_loss: 1.8850 - val_accuracy: 0.4211
Epoch 6/50
5/5 [==============================] - 0s 13ms/step - loss: 0.1665 - accuracy: 0.9311 - val_loss: 1.6527 - val_accuracy: 0.4386
Epoch 7/50
5/5 [==============================] - 0s 15ms/step - loss: 0.1599 - accuracy: 0.9340 - val_loss: 1.4695 - val_accuracy: 0.4561
Epoch 8/50
5/5 [===========

In [12]:
model.summary()

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_189 (Dense)            (None, 66)                2046      
_________________________________________________________________
activation_136 (Activation)  (None, 66)                0         
_________________________________________________________________
batch_normalization_116 (Bat (None, 66)                264       
_________________________________________________________________
dense_190 (Dense)            (None, 49)                3283      
_________________________________________________________________
activation_137 (Activation)  (None, 49)                0         
_________________________________________________________________
batch_normalization_117 (Bat (None, 49)                196       
_________________________________________________________________
dense_191 (Dense)            (None, 2)               

In [13]:
y_preds = model.predict(X_test).argmax(axis=1)
y_preds.shape

(171,)

In [16]:
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix
#np.sqrt(mean_squared_error(y_test, y_preds))
print(classification_report(y_test, y_preds))
print(confusion_matrix(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       108
           1       0.93      0.89      0.91        63

    accuracy                           0.94       171
   macro avg       0.94      0.93      0.93       171
weighted avg       0.94      0.94      0.94       171

[[104   4]
 [  7  56]]
